In [17]:
import pandas as pd
import datetime
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math as m

In [18]:
# lecture des donnees gtfs

df_stops_bus = pd.read_csv('gtfs_bus/stops.txt')
df_routes_bus = pd.read_csv('gtfs_bus/routes.txt')
df_trips_bus = pd.read_csv('gtfs_bus/trips.txt')
df_stop_times_bus = pd.read_csv('gtfs_bus/stop_times.txt')

df_stops_tram = pd.read_csv('gtfs_tram/stops.txt')
df_routes_tram = pd.read_csv('gtfs_tram/routes.txt')
df_trips_tram = pd.read_csv('gtfs_tram/trips.txt')
df_stop_times_tram = pd.read_csv('gtfs_tram/stop_times.txt')

In [50]:
def edges_build(stop_ids_list):
    for x in stop_ids_list:
        x = str(x)
    l = len(stop_ids_list)-1
    i = 0
    edges_list = list()
    while (i<l):
        edges_list.append((str(stop_ids_list[i]), str(stop_ids_list[i+1])))
        i = i+1
    return edges_list
def route_id_to_name(route_id):
    return df_routes[df_routes["route_id"] == route_id]["route_short_name"].tolist()[0]
def stop_to_route(stop_name):
    stop_id = stop_name_to_id(stop_name)["stop_id"].tolist()[0]
    trip_id = df_stop_times[df_stop_times["stop_id"] == int(stop_id)]["trip_id"].tolist()[0]
    route_id = df_trips[df_trips["trip_id"] == trip_id]["route_id"].tolist()[0]
    return route_id_to_name(route_id)

def stop_id_to_name_tram(stop_id):
    return df_stops_tram[df_stops_tram["stop_id"] == stop_id]["stop_name"].tolist()[0]
def stop_name_to_id_tram(stop_name):
    return df_stops_tram[df_stops_tram["stop_name"] == stop_name]["stop_id"].tolist()

def stop_id_to_name_bus(stop_id):
    return df_stops_bus[df_stops_bus["stop_id"] == stop_id]["stop_name"].tolist()[0]
def stop_name_to_id_bus(stop_name):
    return df_stops_bus[df_stops_bus["stop_name"] == stop_name]["stop_id"].tolist()

def stop_ids_list_to_name_bus(ids_list):
    names_list = list()
    for elt in ids_list:
        names_list.append(stop_id_to_name_bus(str(elt))["stop_name"].tolist()[0])
    return names_list

def same_stop(stop_1, stop_2):
    return 0

def distance (lat_1, lon_1, lat_2, lon_2):
    delta = lon_2 - lon_1
    d = np.arccos(np.sin(lat_1)*np.sin(lat_2)+np.cos(lat_1)*np.cos(lat_2)*np.cos(delta))
    return d


def remove_stops(g,stops_list):
    new_graph=g.copy()
    new_graph.remove_nodes_from(stops_list)
    return new_graph

# renvoie les ids des arrets de meme nom    
    
         

In [20]:
#x = 6378137 * distance(stops["3718"].lat, stops["3718"].lon, stops["3759"].lat, stops["3759"].lon)
R = 6373000
lat_1 = stops["3718"].lat
lon_1 = stops["3718"].lon
lat_2 = stops["3759"].lat
lon_2 = stops["3759"].lon
dlon = lon_2-lon_1
dlat = lat_2-lat_1
a =np.sin(dlat/2)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(dlon/2)**2
c = 2 * m.atan2(m.sqrt(a), m.sqrt(1-a))
distance = R * c
print lat_1
print lon_1
print lat_2
print lon_2
print distance

44.83733
-0.577188
44.841056
-0.578464
24335.470034


In [21]:
# construction dictionnaire d'arrets


class Stop:
    def __init__(self):
        self.iden = "UNDEFINED"
        self.name = "UNDEFINED"
        self.lon = 1000.0
        self.lat = 1000.0 

ids = pd.concat([df_stops_bus["stop_id"], df_stops_tram["stop_id"]]).tolist()        
stops = dict()

for elt in ids:
    stops[elt]=Stop()
    df_temp_bus = df_stops_bus[df_stops_bus["stop_id"] == elt]
    df_temp_tram = df_stops_tram[df_stops_tram["stop_id"] == elt]
    if(len(df_temp_bus) > 0):
        stops[elt].iden = df_temp_bus["stop_id"].tolist()[0]
        stops[elt].lat = df_temp_bus["stop_lat"].tolist()[0]
        stops[elt].lon = df_temp_bus["stop_lon"].tolist()[0] 
        stops[elt].name = df_temp_bus["stop_name"].tolist()[0]
    if(len(df_temp_tram) > 0):
        stops[elt].iden = df_temp_tram["stop_id"].tolist()[0]
        stops[elt].lat = df_temp_tram["stop_lat"].tolist()[0]
        stops[elt].lon = df_temp_tram["stop_lon"].tolist()[0] 
        stops[elt].name = df_temp_tram["stop_name"].tolist()[0]
        

In [22]:
# suppression des repetitions dans trips et stop_times
df_trips_bus = df_trips_bus.drop_duplicates(["route_id", "direction_id", "trip_headsign"] )
df_stop_times_bus = df_stop_times_bus[df_stop_times_bus["trip_id"].isin(df_trips_bus["trip_id"])]
df_trips_tram = df_trips_tram.drop_duplicates(["route_id", "direction_id", "trip_headsign"] )
df_stop_times_tram = df_stop_times_tram[df_stop_times_tram["trip_id"].isin(df_trips_tram["trip_id"])]

In [25]:
#Construction du dictionnaire des trips

class Trip:
    def __init__(self):
        self.iden = "UNDEFINED"
        self.route_id = -1
        self.direction_id = -1
        self.trip_headsign = "UNDEFINED"
        self.route_name = "UNDEFINED"
        self.path = list()
        
trips = dict()
for elt in df_trips_bus["trip_id"]:
    trips[elt] = Trip()
    df_bus = df_stop_times_bus[df_stop_times_bus["trip_id"] == elt]
    trips[elt].path = df_bus["stop_id"].tolist()
    for x in trips[elt].path:
        x = str(x)
for elt in df_trips_tram["trip_id"]:
    trips[elt] = Trip()
    df_tram = df_stop_times_tram[df_stop_times_tram["trip_id"] == elt]
    trips[elt].path = df_tram["stop_id"].tolist()  
    for x in trips[elt].path:
        x = str(x)
g = nx.Graph()
g.clear()
for elt in trips:
    g.add_nodes_from(trips[elt].path)
    g.add_edges_from(edges_build(trips[elt].path), trip=elt)

for elt in stops:
    l = stop_name_to_id_tram(stops[elt].name)
    for elt_ in l:
        g.add_edge(elt,elt_, trip="a_pied")
    for elt_ in stops:
        if(same_stop(elt,elt_)):
            g.add_edge(elt,elt_,trip="a_pied")

In [29]:
print stop_name_to_id_tram("Hôtel de Ville")
print stop_name_to_id_tram("Palais de Justice")
print stop_name_to_id_tram("Gambetta")
print g.neighbors("3760")

#print g.neighbors("3720")
#print g.neighbors("3718")
#print stop_id_to_name_tram("3720")
nx.shortest_path(g,source="3720",target="3759")



['3718', '3719', '3757', '3758', 'HDVA', 'HDVB']
['3720', '3721', 'PJUST']
['3759', '3760', 'GAMBT']
['3759', '3758', '3762', '17', '2444', 'GAMBT2', 'GAMBT1', '3760', '6447', '6701', 'GAMBT', '3278']
6932


5207

In [58]:
new_g=remove_stops(g,["3718","3719","3757","3758"])
nx.shortest_path(new_g,source="3720",target="3759")
print stop_id_to_name_bus("3858")


Palais de Justice


In [129]:
df_trips_bus[df_trips_bus["trip_id"] == g.edge[4155][3802]['trip']]

,route_id,service_id,trip_id,trip_headsign,block_id,direction_id
14568,20,2016-M20_A-Semaine-02,10008630-2016-M20_A-Semaine-02,BORDEAUX Victoire,1830208,1


In [49]:
test=nx.Graph()
test.clear()
test.add_node(1)
test.add_node(2)
test.add_node(3)
test.add_node(4)
test.add_edge(1,2)
test.add_edge(2,3)
test.add_edge(3,4)
test.add_edge(1,4)
#test.remove_nodes_from([2,3])
test_bis = test.copy()
test_bis.edges()

[(1, 2), (1, 4), (2, 3), (3, 4)]